In [2]:
import refexps
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats
import pickle

dataset_dir = 'Dataset/Push_to_Talk_Corpus/RefExp'
success_dir = 'Dataset/Push_to_Talk_Corpus/Success'

In [13]:
move_level_data = {'alignment e':[], 'alignment p': [], 'coherence e':[], 'coherence p': []}

for dialogue in listdir(dataset_dir):
    success_file = dialogue.replace('refexps', 'movesuccess')
    success_file = path.join(success_dir, success_file)
    dialogue = path.join(dataset_dir, dialogue)
    #load referring expressions data
    dialogue = refexps.load_data(dialogue)
    success_moves = refexps.load_data(success_file)
    dialogue_move = refexps.move_level(dialogue)
    for move in dialogue_move:
        refs = move[1]
        success = success_moves[move[0]]
        if success == 'no_video':
            continue
        for obj in refs:
            align_speaker = {}
            total_n = len(refs[obj])
            sameform_n =0
            ignore = 0
            entropy_speaker = defaultdict(int)
            total_n_speaker = defaultdict(int)
            sameform_n_speaker = defaultdict(int)
            ignore_speaker = defaultdict(int)
            obj_forms_speaker = defaultdict(dict)
            obj_forms = defaultdict(int)
            for ref in refs[obj]:
                obj_forms[ref[3]] += 1
                speaker = ref[1]
                if not obj_forms_speaker[speaker].has_key(ref[3]):
                    obj_forms_speaker[speaker][ref[3]] = 0
                obj_forms_speaker[speaker][ref[3]] += 1
                align_speaker[speaker] = 0
                total_n_speaker[speaker] +=1
                if refexps.get_previous(ref, refs[obj]) == 'no_previous':
                        ignore += 1
                        ignore_speaker[speaker] += 1
                else: 
                    if ref[3] == refexps.get_previous(ref, refs[obj])[3]:
                        sameform_n +=1
                        sameform_n_speaker[speaker] += 1
            sum_forms = sum(obj_forms.values())
            for form in obj_forms:
                obj_forms[form] = float(obj_forms[form])/float(sum_forms)
            obj_forms = obj_forms.values()
            variation = stats.entropy(obj_forms)
            #movelevel variation and alignment (for each object)
            ntotal_n = total_n - ignore
            for s in align_speaker:
                ntotal_n = total_n_speaker[s] - ignore_speaker[s]
                if ntotal_n != 0:
                    align = float(sameform_n_speaker[s]) / ntotal_n 
                    move_level_data['alignment '+ s.replace('-utts', '')].append((align, success))
            variation_speaker = {}
            for s in obj_forms_speaker:
                sum_forms = sum(obj_forms_speaker[s].values())
                for form in obj_forms_speaker[s]:
                    obj_forms_speaker[s][form] = float(obj_forms_speaker[s][form])/float(sum_forms)
                obj_forms_speaker[s] = obj_forms_speaker[s].values()
                variation_speaker[s] = stats.entropy(obj_forms_speaker[s])
            for s in variation_speaker:
                move_level_data['coherence '+ s.replace('-utts','')].append((variation_speaker[s], success))
            
pickle.dump(move_level_data, open('regression_form.p', 'w'))

{'alignment e': [(0.0, 'incorrect'), (0.0, 'correct'), (0.0, 'correct'), (0.0, 'not_moved'), (0.0, 'not_moved'), (0.0, 'correct'), (0.0, 'incorrect'), (0.0, 'incorrect'), (0.0, 'not_moved'), (0.0, 'not_moved'), (0.0, 'not_moved'), (0.0, 'not_moved'), (1.0, 'not_moved'), (0.0, 'correct'), (0.0, 'correct'), (1.0, 'not_moved'), (1.0, 'correct'), (0.0, 'correct'), (0.0, 'correct'), (1.0, 'correct'), (0.0, 'incorrect'), (1.0, 'incorrect'), (1.0, 'incorrect'), (0.0, 'not_moved'), (0.0, 'not_moved'), (1.0, 'not_moved'), (0.0, 'incorrect'), (0.0, 'correct'), (0.0, 'incorrect'), (0.0, 'incorrect'), (1.0, 'incorrect'), (0.0, 'incorrect'), (0.0, 'not_moved'), (0.0, 'not_moved'), (0.0, 'not_moved'), (0.6666666666666666, 'incorrect'), (0.5, 'incorrect'), (0.0, 'incorrect'), (1.0, 'not_moved'), (1.0, 'not_moved'), (1.0, 'correct'), (1.0, 'incorrect'), (0.0, 'not_moved'), (1.0, 'not_moved'), (1.0, 'not_moved'), (0.0, 'not_moved'), (0.0, 'not_moved'), (1.0, 'not_moved'), (0.0, 'not_moved'), (0.0, 'not